# Deploying Agents to Amazon Bedrock Agent Core Runtime (Optional)

If you've made it this far you've already built a strands agent, used tools from an MCP server, and successfully interacted with it to verify its calling tools using Amazon Nova.

In this notebook, we'll deploy our customer support agent to **AWS Bedrock Agent Core Runtime** - a serverless platform designed for AI agents and tools.

## What is AWS Bedrock Agent Core Runtime?

Agent Core Runtime provides:
- **Serverless deployment** - No infrastructure management
- **Secure isolation** - Each session runs in its own environment
- **MCP protocol support** - Native support for Model Context Protocol
- **Auto-scaling** - Handles traffic spikes automatically
- **Framework agnostic** - Works with any agent framework

Learn more: [Agent Core Documentation](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/agents-tools-runtime.html)

## AgentCore Architecture for AnyCo Telecom Customer Support Agent
Because our relational database must exist in a VPC, we've already created the security groups and subnet IDs required for deploy our AgentCore Runtime inside our VPC.

We'll leverage the [AgentCore CLI](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/runtime-get-started-toolkit.html) to make it easy to deploy to our VPC

![AgentCore Agent Architecture](../images/workshop_diagrams-agentcore.drawio.png)

## Step 1: Install required modules

In [ ]:
%pip install -r requirements.txt --quiet

## Step 2: Examine Our Enhanced MCP Server

Let's look at our revised MCP file that we'll be deploying to Agent Core Runtime. This version includes a number of tools we can use to interact with our relational database.

In [15]:
from IPython.display import display, HTML, Markdown

# Display the key components of our MCP server
with open('telecom_mcp_agentcore_rt.py', 'r') as f:
    code = f.read()

# Show just the first part to understand the structure
lines = code.split('\n')
preview_lines = lines[:80]  # Show first 80 lines
preview_code = '\n'.join(preview_lines)

display(Markdown(f"```python\n{preview_code}\n```"))

```python
"""
Telecom MCP Server for Amazon Bedrock AgentCore Runtime
Implements telecom customer support functionality with AgentCore integration
"""

import sys
import logging
import os
from typing import Dict, Any
from datetime import datetime, timezone

# database imports
import pandas as pd
from sqlalchemy import create_engine, text

# AgentCore and Strands framework imports
from strands import Agent, tool
from strands_tools.utils.user_input import get_user_input
from strands.agent.conversation_manager import SummarizingConversationManager
from nova_custom_model_provider import NovaCustomModelProvider
from pydantic import BaseModel, Field

from bedrock_agentcore import BedrockAgentCoreApp

# Structured logging configuration
import structlog


# Configure structlog
def add_timestamp(logger, method_name, event_dict):
    """Add timestamp to log entries"""
    event_dict["timestamp"] = datetime.now(timezone.utc).isoformat().replace('+00:00', 'Z')
    return event_dict


def add_service_context(logger, method_name, event_dict):
    """Add service context to log entries"""
    event_dict["service"] = "telecom-mcp-agentcore"
    event_dict["version"] = "1.0.0"
    return event_dict


# Set logging level from environment variable or default to DEBUG for development
LOG_LEVEL = os.getenv("LOG_LEVEL", "DEBUG").upper()

# Configure standard library logging for Strands SDK
logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    stream=sys.stdout,
    level=getattr(logging, LOG_LEVEL, logging.DEBUG),
)

# Configure Strands SDK logger specifically
strands_logger = logging.getLogger("strands")
strands_logger.setLevel(getattr(logging, LOG_LEVEL, logging.DEBUG))

# Configure structlog processors
structlog.configure(
    processors=[
        add_timestamp,
        add_service_context,
        structlog.stdlib.filter_by_level,
        structlog.stdlib.add_logger_name,
        structlog.stdlib.add_log_level,
        structlog.stdlib.PositionalArgumentsFormatter(),
        structlog.processors.StackInfoRenderer(),
        structlog.processors.format_exc_info,
        structlog.processors.UnicodeDecoder(),
        structlog.processors.JSONRenderer(),
    ],
    context_class=dict,
    logger_factory=structlog.stdlib.LoggerFactory(),
    wrapper_class=structlog.stdlib.BoundLogger,
    cache_logger_on_first_use=True,
)

# Create structured logger for application code
logger = structlog.get_logger(__name__)

# Initialize BedrockAgentCore app
```

## Step 3: Create Agent Core Execution Role

Before deploying, we need to create an **execution role** that grants our agent permissions to:
- Invoke Bedrock models  
- Write logs to CloudWatch
- Access ECR repositories

In [ ]:
from utils import create_agent_core_execution_role

# Create execution role for our restaurant booking agent
agent_core_role_arn = create_agent_core_execution_role(
    agent_name='telecom_cust_support'
)

print(f"Agent Core Role ARN: {agent_core_role_arn}")

# Store for use in subsequent cells
%store agent_core_role_arn

## Step 4: Configure Agent Core Deployment

Now we'll configure our deployment using the AgentCore CLI. This will:
1. Create configuration files (`.bedrock_agentcore.yaml` and `Dockerfile`)
2. Set up ECR repository for container storage
3. Configure the execution role and dependencies
4. Kick off the build of the container to deploy to AgentCore Runtime.

**Note**: the variable `agent_core_role_arn` should already be loaded from above cells. Otherwise, replace with your role ARN for agent core.

In [ ]:
# Define variables for VPC networking details
from utils import extract_CF_outputs
BR_AGENTCORE_SG, BR_AGENTCORE_SUBNETS = extract_CF_outputs("BedrockAgentCoreSecurityGroup", "BedrockAgentCoreSubnets")

## Notes on version for AgentCore CLI

VPC support for the AgentCore CLI was added in version `bedrock-agentcore-starter-toolkit==0.1.32` so be sure to pip install that version if you run into any issues: 

`pip install bedrock-agentcore-starter-toolkit>=0.1.32`




### Configuring our deployment using AgentCore CLI
To build our configuration files, **run the below cell and copy and paste into a terminal session**. Note that this first changes directory to this local directory and assumes you're using sagemaker studio. If you're running this on your own, uncomment the below cell to run `!pwd` and see your current working directory, and `cd` into that instead.

In [ ]:
# show current working directory
# !pwd

In [ ]:
import os
print("Run this command in the terminal within this directory:\n\n", f'''cd /home/sagemaker-user/labmodules/3-agentic-workflow && \\
    agentcore configure -e telecom_mcp_agentcore_rt.py \\
    --execution-role {agent_core_role_arn} \\
    --region "{os.environ['AWS_REGION']}" \\
    --requirements-file "requirements.txt" \\
    --disable-memory \\
    --vpc \\
    --subnets {BR_AGENTCORE_SUBNETS} \\
    --security-groups {BR_AGENTCORE_SG} \\
    --non-interactive \\
    --authorizer-config null \\
    --ecr "auto"''')

You should see something similar to this print to the terminal console, indicating the configuration file has been successfully created:

```bash
Generated .dockerignore
Generated Dockerfile: .bedrock_agentcore/telecom_mcp_agentcore_rt/Dockerfile
Setting 'telecom_mcp_agentcore_rt' as default agent


╭──────────────── Configuration Success ────────────────╮
│ Agent Details                                         │
│ Agent Name: telecom_mcp_agentcore_rt                  │
│ Runtime: None                                         │
│ Region: us-east-1                                     │
│ Account: <account_id>                                 │
│                                                       │
│ Configuration                                         │
│ Execution Role: arn:aws:iam::<account_id>:role/       │
│   AgentCoreExecutionRole_restaurant_booking           │
│ ECR Repository: Auto-create                           │
│ Authorization: IAM (default)                          │
│                                                       │
│ Memory: Disabled                                      │
│                                                       │
│ 📄 Config saved to: /home******/amazon-nova-          │
│   samples/multimodal-understanding/workshop/          │
│   4-agentic-workflow/.bedrock_agentcore.yaml          │
│                                                       │
│ Next Steps:                                           │
│    agentcore launch                                   │
╰───────────────────────────────────────────────────────╯
```


## Step 5: Deploy to Agent Core Runtime

Now we'll deploy our agent to the cloud! This uses **CodeBuild** for cloud-based container building (no local Docker required).

The deployment process:
1. **Package** application with dependencies
2. **Build** container image using CodeBuild
3. **Push** to ECR repository
4. **Deploy** to Agent Core Runtime
5. **Create** runtime endpoints

In [ ]:
# Deploy to Agent Core Runtime using CodeBuild
!agentcore launch --auto-update-on-conflict

### Potential AgentCore CLI permissions errors

Sometimes AgentCore CLI will attempt to use IAM roles that aren't ready to be used yet. You may see an error such as:
```
Launch failed: Launch failed. Resources created: ['ECR Repository: account-id.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-telecom_mcp_agentcore_rt', 'Runtime Execution Role: arn:aws:iam::account-id:role/AgentCoreExecutionRole_telecom_cust_support', 'CodeBuild Execution Role: arn:aws:iam::account-id:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-f2323f']
```

Give this a few moments, and try running `agentcore launch` again, to make sure the permissions have been created completely.

## Step 6: Monitor Your Deployment

Check the status of your deployment and get information about logs. Once the deployment is ready, you should see the following: `Ready - Agent deployed and endpoint available`

In [ ]:
# Check deployment status and get monitoring information
!agentcore status

## Step 7: Test Our Deployed Agent

🎉 **Success!** Your AgentCore Runtime endpoint is now up and running. Let's test it by asking about an account as if we were a customer.

In [ ]:
agent_arn = '<agent core arn from above>'
prompt = "Hi, I'm calling about my account. My phone number is 555-1001. Can you tell me what plan I'm on and how much data I've used this month?"

import boto3
import json

# Initialize the Bedrock AgentCore client
agent_core_client = boto3.client('bedrock-agentcore')
  
# Prepare the payload
payload = json.dumps({"input": prompt}).encode()
  
# Invoke the agent
response = agent_core_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    payload=payload
)
  
# Process and print the response
if "text/event-stream" in response.get("contentType", ""):
    # Handle streaming response
    content = []
    for line in response["response"].iter_lines(chunk_size=10):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    print("\nComplete response:", "\n".join(content))

elif response.get("contentType") == "application/json":
    # Handle standard JSON response
    content = []
    for chunk in response.get("response", []):
        content.append(chunk.decode('utf-8'))
    print(json.loads(''.join(content))['content'][0]['text'])
  
else:
    # Print raw response for other content types
    print(response)

zsh:1: unmatched "


### Understanding the Status Output:

The status shows:
- **Agent Info**: Name, ID, ARN, timestamps
- **Configuration**: Region, account, execution role, ECR repository
- **Endpoint Status**: Runtime endpoint status and availability
- **Logs**: CloudWatch log locations for debugging

### Viewing Logs:
```bash
aws logs tail /aws/bedrock-agentcore/runtimes/YOUR-AGENT-ID --follow
```

## Step 8: Clean Up Resources

⚠️ **Important**: Clean up resources to avoid unnecessary charges, and run the following command in your terminal. This will permanently delete your agent deployment, and be sure to say "Y" (Yes) at the prompt.

```bash
cd /home/sagemaker-user/labmodules/3-agentic-workflow && agentcore destroy
```

## 🎯 Summary

Congratulations! You have successfully:

✅ **Built a customer support Agent** using Strands
✅ **Deployed an MCP Server** to AWS Bedrock Agent Core Runtime  
✅ **Configured Security** with proper IAM roles and permissions  
✅ **Tested Cloud Deployment** with customer service operations  
✅ **Monitored Your Agent** using status and logging capabilities  
✅ **Cleaned Up Resources** to avoid unnecessary costs  

## 🔑 Key Learnings

### Agent Core Runtime Benefits:
- **Serverless**: No infrastructure management
- **Secure**: Isolated execution environments
- **Scalable**: Auto-scaling based on demand
- **Protocol Support**: Native MCP integration

### Deployment Process:
1. **Configure**: Set parameters and security
2. **Build**: Create containers (CodeBuild recommended)
3. **Deploy**: Push to runtime with auto-scaling
4. **Monitor**: Track performance via CloudWatch

## 📚 Additional Resources

- **[Agent Core Developer Guide](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/)**: Complete documentation
- **[MCP Deployment Guide](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/runtime-mcp.html)**: MCP-specific deployment info
- **[Runtime Architecture](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/runtime-how-it-works.html)**: Understanding the platform

## 🚀 Next Steps

Consider these next steps for your learning journey:

### 1. **What could go wrong here?**
- Would we really want to put SQL statements directly in our Agent Tools?
- How are we ensuring the agent inherits the correct row level security that the user has?
- How are we protecting against spoofing, and acting like someone else?

### 2. **Production Features**
- Set up CI/CD pipelines
- Implement monitoring and alerting
- Configure auto-scaling policies

### 3. **Explore Agent Core Ecosystem**
- Try Agent Core Gateway for API integration
- Experiment with Agent Core Browser for web automation
- Build multi-agent systems with agent-to-agent communication

### 4. **Advanced Patterns**
- Implement streaming responses
- Add memory capabilities
- Create complex workflow orchestration

Great work on completing this deployment workshop! 🎉